# Tree모델 돌리기

### 1. 데이터 준비

In [1]:
import pandas as pd
import numpy as np

df= pd.read_csv('train1.csv')
df_test = pd.read_csv('test1.csv')
df.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,sqft_lot15,total_rooms,sqft_ratio,sqft_total_size,sqft_ratio_1,sqft_ratio15,is_renovated,per_price,var,mean
0,0,0,20141013,12.309987,3,1.00,7.074117,8.639588,1.0,0,...,5650,4.00,0.818803,7.074117,1.0,0.237168,0,1.740145,0.171641,1.324285
1,1,1,20150225,12.100718,2,1.00,6.647688,9.210440,1.0,0,...,8062,3.00,0.721756,6.647688,1.0,0.337385,0,1.820290,0.153971,1.392626
2,2,2,20150218,13.142168,3,2.00,7.427144,8.997271,1.0,0,...,7503,5.00,0.825489,7.427144,1.0,0.239904,0,1.769478,0.111248,1.522484
3,3,3,20140627,12.458779,3,2.25,7.447751,8.827615,2.0,0,...,6819,5.25,0.843688,7.447751,1.0,0.328201,0,1.672824,0.135985,1.426763
4,4,4,20150115,12.583999,3,1.50,6.966967,9.181118,1.0,0,...,9711,4.50,0.758837,6.966967,1.0,0.169910,0,1.806238,0.143851,1.439939


In [2]:
#train, test나누기

from sklearn.model_selection import train_test_split

train_data = df.drop(['price'], axis=1)
target_data = df['price']

x_train, x_test, y_train, y_test = train_test_split(train_data, target_data, random_state=1, test_size=0.2)

#train set을 8:2로 다시 나누기(validation set)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=1, test_size=0.2)

print('Entire data' , train_data.shape)
print('train set' , x_train.shape, y_train.shape)
print('valid set' , x_valid.shape, y_valid.shape)
print('test set' , x_test.shape, y_test.shape)


Entire data (15029, 30)
train set (9618, 30) (9618,)
valid set (2405, 30) (2405,)
test set (3006, 30) (3006,)


### 2. DecisionTreeRegressor

In [3]:
from sklearn.tree import DecisionTreeRegressor

tr = DecisionTreeRegressor(random_state=0)
tr = tr.fit(x_train, y_train)

tr_prediction = tr.predict(x_test)
print(tr_prediction)

[12.84792916 13.21585554 13.25339339 ... 12.46837665 12.86618256
 12.76554842]


In [4]:
#csv 파일로 만들기(제출용)

submit_tr = pd.DataFrame({
    'Id' : x_test['id'],
    'Price' : tr_prediction
})
submit_tr.to_csv('submit_tr.csv', index=False)
submit_tr.head()

,Id,Price
7929,7934,12.847929
8095,8100,13.215856
13412,13418,13.253393
5869,5873,12.765691
1955,1956,12.765620


### 3. Extra tree Regressor


In [5]:
from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=100, random_state=0)
etr = tr.fit(x_train, y_train)

etr_prediction = tr.predict(x_valid)
print(etr_prediction)

[13.21767538 13.59236826 13.00583181 ... 13.74723279 12.89922233
 12.73670384]


### 4. Random Forest

In [6]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(x_train, y_train)

rf_prediction = rf.predict(x_valid)
print(rf_prediction)

[13.57495009 13.58023916 12.98551325 ... 13.64841186 12.88281997
 12.89492845]


### 5. GradientBoostingRegressor

In [7]:
from sklearn.ensemble import GradientBoostingRegressor

# n_estimators=10 : 트리개수 #max_depth : 트리깊이 #learning_rate : 매개변수 각 트리 기여 정도

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=10, learning_rate=1.0)
gbr.fit(x_train, y_train)

gbr_prediction = gbr.predict(x_valid)
print(gbr_prediction)

[13.48577522 13.70473072 13.14860391 ... 13.30727665 13.04130077
 12.97602773]


In [8]:
#최적의 트리수 찾기

import numpy as np
from sklearn.metrics import mean_squared_error

errors = [mean_squared_error(y_valid, y_test)
         for y_test in gbr.staged_predict(x_valid)]

best_estimators = np.argmin(errors)

gbr_best = GradientBoostingRegressor(max_depth=2, n_estimators=best_estimators)
gbr_best.fit(x_train, y_train)



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=9,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### 교차 검증

In [9]:
#GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
scores = cross_val_score(gbr, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("scores:", scores)
    print("Mean:", scores.mean())
    print("sd:", scores.std())
    
display_scores(tree_rmse_scores)

scores: [0.22705028 0.21553253 0.22574379 0.21648395 0.20305111 0.23388252
 0.23039976 0.22533752 0.22381639 0.22213243]
Mean: 0.222343026550511
sd: 0.008341857951358066


In [10]:
#Decision tree regressor

from sklearn.model_selection import cross_val_score
scores = cross_val_score(tr, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("scores:", scores)
    print("Mean:", scores.mean())
    print("sd:", scores.std())
    
display_scores(tree_rmse_scores)

scores: [0.17434332 0.18503678 0.18371628 0.16648251 0.17037943 0.19249187
 0.17028484 0.18686053 0.19470254 0.18322377]
Mean: 0.18075218713239363
sd: 0.009299748824983384


In [11]:
#Extra trees regressor

from sklearn.model_selection import cross_val_score
scores = cross_val_score(etr, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("scores:", scores)
    print("Mean:", scores.mean())
    print("sd:", scores.std())
    
display_scores(tree_rmse_scores)

scores: [0.17434332 0.18503678 0.18371628 0.16648251 0.17037943 0.19249187
 0.17028484 0.18686053 0.19470254 0.18322377]
Mean: 0.18075218713239363
sd: 0.009299748824983384


In [12]:
#Random forest

from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("scores:", scores)
    print("Mean:", scores.mean())
    print("sd:", scores.std())
    
display_scores(tree_rmse_scores)

scores: [0.12197025 0.11593468 0.13037086 0.11525481 0.11125026 0.12550622
 0.11919303 0.12213945 0.12017168 0.11727162]
Mean: 0.11990628701300862
sd: 0.005195871471235793


### 하이퍼파라미터 찾기(GridSearch 이용)

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]},
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, 
                          scoring='neg_mean_squared_error', 
                          return_train_score=True)

grid_search.fit(x_train, y_train)

prediction = grid_search.predict(x_valid)
print(prediction)


[13.49339626 13.5997739  12.90853463 ... 13.61077617 12.89318763
 12.97726064]


In [14]:
prediction = grid_search.predict(x_valid)
print(prediction)


[13.49339626 13.5997739  12.90853463 ... 13.61077617 12.89318763
 12.97726064]


In [15]:
# 특성의 상대적 중요도 파악

feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances



array([0.00392759, 0.00343134, 0.0049036 , 0.00157837, 0.02408105,
       0.1950579 , 0.01033725, 0.00249196, 0.00204092, 0.00971156,
       0.00295911, 0.11702857, 0.04792931, 0.00605788, 0.0119788 ,
       0.00858086, 0.013671  , 0.22371303, 0.01776408, 0.06112228,
       0.00808354, 0.01494849, 0.01344894, 0.06425692, 0.00449012,
       0.01040319, 0.00063864, 0.06159694, 0.02238389, 0.03138286])

### RandomizedSearchCV 이용

In [16]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(
#               estimator = rf, 
#               param_distributions = random_grid, 
#               n_iter = 100, 
#               cv = 3, 
#               verbose = False,
#               scoring = make_scorer(precision_score),
#               random_state=42, n_jobs = -1)

# rf_random.fit(x_train, y_train)



## 위도, 경도, zipcode